<a href="https://colab.research.google.com/github/visha1Sagar/Multi-LLM-RAG-Powered-QA-Bot/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing required libraries

In [1]:
!pip install --quiet --upgrade langchain langchain-community gradio pypdf langchain-openai faiss-cpu pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.5 

## loading credentials from .env

In [ ]:
# import os
# from dotenv import load_dotenv

# load_dotenv('azure_credentials.env')

# EMBEDDING_MODEL_NAME = os.getenv("EMBEDDING_MODEL_NAME")
# EMBEDDING_ENDPOINT = os.getenv("EMBEDDING_ENDPOINT")
# EMBEDDING_API_VERSION = os.getenv("EMBEDDING_API_VERSION")
# EMBEDDING_API_KEY = os.getenv("EMBEDDING_API_KEY")
# AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
# API_VERSION = os.getenv("API_VERSION")
# AZURE_ENDPOINT = os.getenv("AZURE_ENDPOINT")

## From Colab Keys

In [9]:
from google.colab import userdata

EMBEDDING_MODEL_NAME = userdata.get("EMBEDDING_MODEL_NAME")
EMBEDDING_ENDPOINT = userdata.get("EMBEDDING_ENDPOINT")
EMBEDDING_API_VERSION = userdata.get("EMBEDDING_API_VERSION")

AZURE_OPENAI_API_KEY = userdata.get("AZURE_OPENAI_API_KEY")

API_VERSION = userdata.get("api_version")
AZURE_ENDPOINT = userdata.get("AZURE_ENDPOINT")

API_VERSION_4o = userdata.get("api_version_4o")
AZURE_ENDPOINT_4o = userdata.get("AZURE_ENDPOINT_4o")

## Importing required libraries

In [3]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores import FAISS

from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.runnables import RunnableParallel

from langchain_core.prompts import ChatPromptTemplate

from langchain_community.document_loaders import PyPDFLoader

from langchain_google_genai import ChatGoogleGenerativeAI

from utils import *

## Defining LLMs

In [10]:
llm = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY  ,
    api_version=API_VERSION_4o,
    azure_endpoint = AZURE_ENDPOINT_4o
    )

llm_2 = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY  ,
    api_version=API_VERSION,
    azure_endpoint = AZURE_ENDPOINT
    )


In [11]:
embeddings = AzureOpenAIEmbeddings(
    model=EMBEDDING_MODEL_NAME,
    azure_endpoint= EMBEDDING_ENDPOINT,
    api_key= AZURE_OPENAI_API_KEY,
    openai_api_version= EMBEDDING_API_VERSION
)

## Prompts

In [12]:
from utils import prompt_ as rag_prompt, simple_prompt

## Getting Vector DB's Retriever

In [13]:
retriever = await get_retriever(["book.pdf"], embeddings=embeddings)

## Chains

In [26]:
from utils import get_rag_chain


# Define the second chain with LLM 2
rag_chain_1 = get_rag_chain(retriever, llm_2, rag_prompt)## GPT 4o Mini with Rag
rag_chain_2 = get_rag_chain(retriever, llm, rag_prompt)         ## 4o with Rag

rag_imp = get_rag_chain_custom(retriever, llm, llm_2, rag_prompt)

# Define the third chain with LLM, without RAG
chain3 = (                                                        ## 4o Raw
    simple_prompt
    | llm  # Second LLM instance
    | StrOutputParser()
)


# Run all chains in parallel with RunnableParallel
parallel_chain = RunnableParallel(
    {"miniRag": rag_chain_1, "GPTRag": rag_chain_2,
     "GPT": chain3,  "Rag_exp":rag_imp}
)


# Gradio

In [18]:
import gradio as gr

# List for storing prev asked questions

In [19]:
import csv
import os


In [20]:
def write_or_append_to_csv(file_name, data, headers=None):

    file_exists = os.path.exists(file_name)

    with open(file_name, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        # If the file does not exist, write headers (if provided)
        if not file_exists and headers:
            writer.writerow(headers)

        # Write the data rows
        writer.writerows(data)

In [23]:
def save_history(question, resp1):
    file_name = "data.csv"
    headers = ["questions"]+list(resp1.keys())

    write_or_append_to_csv(file_name, [[question]+ list(resp1.values())], headers)


In [24]:
# Function to call the chains and storing the answers in list.
def generate_response(message, _):
    global Questions_Answers
    responses = parallel_chain.invoke(message)



    save_history(message, responses)

    return responses["GPTRag"], responses["GPT"], responses["miniRag"], responses["Rag_exp"]

In [25]:
def search_history(search_term,file_name = "data.csv"):

    if not os.path.exists(file_name):
        raise FileNotFoundError(f"The file '{file_name}' does not exist.")

    filtered_results = []

    # Read the CSV and search for the term
    with open(file_name, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        headers = next(reader, None)  # Read headers if they exist

        for row in reader:
            # Assume the question is in the first column
            if search_term.lower() in row[0].lower():
                filtered_results.append(tuple(row))

    return filtered_results

### Gradio Interface

In [28]:

# Gradio Interface for Chatbot Tab
with gr.Blocks(fill_width=True, fill_height=True) as chatbot_tab:
    gr.Markdown("### Chatbot")

    # Input box for user question
    question_input = gr.Textbox(label="Ask a question")

    # Button to submit question
    submit_button = gr.Button("Submit")

    gr.HTML("<hr>")
    gr.Markdown("**Noobie Rag**")
    response4 = gr.Markdown("")

    gr.HTML("<hr>")
    gr.Markdown("**GPT Rag**")
    response1 = gr.Markdown("")


    gr.HTML("<hr>")
    gr.Markdown("**GPT Simple**")
    response2 = gr.Markdown("")

    gr.HTML("<hr>")
    gr.Markdown("**MINI Rag**")
    response3 = gr.Markdown("")

    gr.HTML("<hr>")


    # Connect question input to chatbot function and display response
    submit_button.click(fn=generate_response,
                        inputs=question_input, outputs=[response1, response2, response3, response4])


# Gradio Interface for Previous Responses Tab
with gr.Blocks() as history_tab:
    gr.Markdown("### Previous Responses")

    # Search bar to filter questions
    search_input = gr.Textbox(label="Search questions")

    # Display previous questions in rows
    questions_output = gr.Dataframe(headers=["Question", "Response"], interactive=False)

    # Update questions output based on search term
    search_input.change(fn=search_history, inputs=search_input, outputs=questions_output)


# Combine the tabs into a single interface
with gr.Blocks() as app:
    with gr.Tabs():
        with gr.TabItem("Chatbot"):
            chatbot_tab.render()

        with gr.TabItem("Previous Responses"):
            history_tab.render()

/usr/local/lib/python3.10/dist-packages/gradio/utils.py:999: UserWarning: Expected 2 arguments for function <function generate_response at 0x7a8b63e75990>, received 1.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:1003: UserWarning: Expected at least 2 arguments for function <function generate_response at 0x7a8b63e75990>, received 1.
  warnings.warn(


In [29]:
app.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1e91f155ade4933694.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
